In [7]:
import cv2
import time
import numpy as np
import scipy
from scipy import signal
from os import listdir
from os.path import isfile, join
from Emplementation.empl import create_list_distance,compare_hand

def waste_frames(cap,number):
    for i in range(number):
        cap.read()
        
def waste_time(cap):
    waste_frames(cap,0)
    

def getTotalFrames(input_source):
    tempCap = cv2.VideoCapture(input_source)
    ret=0
    while 1:
        hasFrame, frame=tempCap.read()
        if not hasFrame:
            break
        ret+=1
    return ret

protoFile = "hand/pose_deploy.prototxt"
weightsFile = "hand/pose_iter_102000.caffemodel"
nPoints = 21
POSE_PAIRS = [ [0,1],[1,2],[2,3],[3,4],[0,5],[5,6],[6,7],[7,8],[0,9],[9,10],[10,11],[11,12],[0,13],[13,14],[14,15],[15,16],[0,17],[17,18],[18,19],[19,20] ]

threshold = 0.2

In [ ]:
toSave = 2
fileName = "tempUser.txt"
saveString = input("Do you want to create a new user or edit old user?(Y/N):")
if saveString == 'Y' or saveString == 'y' or saveString == 'yes' or saveString == 'Yes':
    toSave = 1
    fileName = input("Enter the unique username:")+".pass"
elif saveString == 'N' or saveString == 'n' or saveString == 'no' or saveString == 'No':
    toSave = 0
else:
    print("Can't take a defined input. Will give output as a list of list of tuples")

Do you want to create a new user or edit old user?(Y/N):y


In [3]:
input_source = input("Enter FileName:")
if not isfile(input_source):
    print(input_source+" is not a valid file.")

totalFrames=getTotalFrames(input_source)
print("Total frames: {}".format(totalFrames))

Enter FileName:3_2.mp4
Total frames: 94


In [4]:
cap = cv2.VideoCapture(input_source)
waste_time(cap)
hasFrame, frame = cap.read()

frameWidth = frame.shape[1]
frameHeight = frame.shape[0]

aspect_ratio = frameWidth/frameHeight

inHeight = 368
inWidth = int(((aspect_ratio*inHeight)*8)//8)

vid_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 15, (frame.shape[1],frame.shape[0]))

net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
k = 0

smoothed_data, data, previous_points, frame_number = [], [], [], 0
for i in range(nPoints): previous_points.append((0,0))
# Smoothing parameters - choose a window length of about 1/2 to 1/4 the fps (so 60 fps --> window length of 33)
window_length, exponent_value = 5,2 # Window length must be odd

In [5]:
### This is going to take some time before the party get's started! The downside of smoothing is that
### data from the past and the future is required to smooth data in the present. This means that all the frames
### in the video must be processed before smoothing the data and displaying the result. This method is therefore
### not suitable for realtime results.
frameSkip = 0
inStart = 1
while 1:
    k+=1
    t = time.time()
    waste_time(cap)
    hasFrame, frame = cap.read()
    frameCopy = np.copy(frame)
    if not hasFrame:
        break

    inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight),
                              (0, 0, 0), swapRB=False, crop=False)

    net.setInput(inpBlob)

    output = net.forward()

    print("forward = {}".format(time.time() - t))

    # Empty list to store the detected keypoints
    points = []
    for i in range(nPoints):
        # confidence map of corresponding body's part.
        probMap = output[0, i, :, :]
        probMap = cv2.resize(probMap, (frameWidth, frameHeight))

        # Find global maxima of the probMap.
        minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

        if prob > threshold :
            cv2.circle(frameCopy, (int(point[0]), int(point[1])), 6, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frameCopy, "{}".format(i), (int(point[0]), int(point[1])), cv2.FONT_HERSHEY_SIMPLEX, .8, (0, 0, 255), 2, lineType=cv2.LINE_AA)

            # Add the point to the list if the probability is greater than the threshold
            points.append((int(point[0]), int(point[1])))
        else :
            # Add the last known point (ex: if thumb is not detected, use thumb position from previous frame)
            points.append(previous_points[i])
            
    # Save the data from the model - data is a list of lists. Each element is a list containing the 22 coordinates for the hand.
    data.append(points)
    
    previous_points = points

    print("total = {}".format(time.time() - t))

forward = 4.824130058288574
total = 4.875565052032471
forward = 4.543867111206055
total = 4.596924543380737
forward = 4.464280366897583
total = 4.51653790473938
forward = 4.935163497924805
total = 5.003743886947632
forward = 4.874606370925903
total = 4.930092811584473
forward = 4.894113779067993
total = 4.946854829788208
forward = 4.657353162765503
total = 4.717630624771118
forward = 5.636996030807495
total = 5.702786445617676
forward = 5.678832054138184
total = 5.74321985244751
forward = 5.741757154464722
total = 5.808627128601074
forward = 5.270529747009277
total = 5.334971904754639
forward = 5.299558401107788
total = 5.364752769470215
forward = 5.17823338508606
total = 5.243131399154663
forward = 5.2794835567474365
total = 5.344367980957031
forward = 5.168941259384155
total = 5.237375497817993
forward = 5.026272296905518
total = 5.082418441772461
forward = 4.3048012256622314
total = 4.354666471481323
forward = 4.09766411781311
total = 4.149636268615723
forward = 4.075172185897827
to

In [6]:
frameSkip=0
data3 = []
for i in data:
    if (0,0) in i:
        frameSkip+=1
    else:
        break
data3 = data[frameSkip:]

In [7]:
# Re-capture the source, so that the video starts at the beginning again
data=data3
cap = cv2.VideoCapture(input_source)
nPoints=21
waste_frames(cap,frameSkip)

# Smooth it out
# Split the data so that just the x values for the first point are made into a list
smoothed_data_in_series = []
for point_index in range(nPoints): # Iterate through each point (wrist, thumb, etc...)
    data_point_series_x = []
    data_point_series_y = []
    for values in data: # Iterate through the series of data (each frame of video)
        data_point_series_x.append(values[point_index][0])
        data_point_series_y.append(values[point_index][1])
    # Smooth the x and y values
    smoothed_data_point_series_x = signal.savgol_filter(data_point_series_x, window_length, exponent_value)
    smoothed_data_point_series_y = signal.savgol_filter(data_point_series_y, window_length, exponent_value)
    smoothed_data_in_series.append(smoothed_data_point_series_x)
    smoothed_data_in_series.append(smoothed_data_point_series_y)
    
# Now the data is sepearted into 44 lists (two lists for each of the 22 locations of the hand, time to zip()
for current_frame_number in range(len(smoothed_data_in_series[0])):
    frame_values = []
    for point_index in range(nPoints):
        x = smoothed_data_in_series[point_index*2][current_frame_number]
        y = smoothed_data_in_series[point_index*2+1][current_frame_number]
        frame_values.append((x,y))
    smoothed_data.append(frame_values)



/usr/local/lib/python3.6/dist-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


In [6]:
create_list_distance(smoothed_data,fileName) #save anyways

if toSave ==2 :
    print(smoothed_data)
else: #if to compare
    files = listdir()
    for file in files:
        if not isfile(file) or not file[-5:] == '.pass' or file == fileName:
            continue
        print(file)
        compare_hand (fileName,file)
    
    


4.pass

 percentage :74.33478471214319
failed
1.pass

 percentage :74.66225530741657
failed
2.pass

 percentage :85.80369451337194
passed
3.pass

 percentage :85.88357588357589
passed


In [9]:
# Iterate through each frame of data
for values in smoothed_data:
    waste_time(cap)
    hasFrame, img = cap.read()

    # When data is smoothed, floats are introduced, these must be eliminated becase cv2.circle requries integers
    values = np.array(values, int)  
    
    # Draw Skeleton
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]
        cv2.line(img, (values[partA][0], values[partA][1]),(values[partB][0], values[partB][1]), (32, 255, 0), 2, lineType=cv2.LINE_AA)
        cv2.circle(img, (values[partA][0], values[partA][1]), 2, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
        cv2.circle(img, (values[partB][0], values[partB][1]), 2, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

    #cv2.imshow('Output-Skeleton', img)
    #cv2.waitKey(0)
    #vid_writer.write(img)

#cv2.destroyAllWindows()
#vid_writer.release()
#cap.release()


In [10]:
print(smoothed_data)

[[(849.9999999999995, 337.2857142857142), (801.8571428571425, 304.9999999999999), (771.1428571428567, 255.85714285714272), (725.085714285714, 241.99999999999986), (664.2857142857142, 212.71428571428552), (693.085714285714, 304.1142857142857), (615.1428571428571, 273.14285714285705), (569.0857142857141, 255.85714285714272), (522.085714285714, 258.00000000000006), (695.2285714285713, 350.8571428571427), (600.142857142857, 338.9999999999998), (552.9428571428572, 351.1142857142855), (492.142857142857, 349.85714285714266), (709.9999999999999, 382.99999999999983), (631.1142857142853, 398.9999999999999), (570.142857142857, 413.9999999999997), (521.9428571428568, 413.85714285714255), (724.9428571428571, 429.9999999999999), (662.1428571428568, 443.85714285714266), (633.2857142857141, 476.9999999999997), (602.2857142857139, 492.08571428571435)], [(849.9999999999995, 333.8571428571427), (809.571428571428, 304.9999999999999), (772.6285714285707, 263.5714285714284), (724.8571428571424, 241.99999999

In [4]:
smoothed_data=[[(849.9999999999995, 337.2857142857142), (801.8571428571425, 304.9999999999999), (771.1428571428567, 255.85714285714272), (725.085714285714, 241.99999999999986), (664.2857142857142, 212.71428571428552), (693.085714285714, 304.1142857142857), (615.1428571428571, 273.14285714285705), (569.0857142857141, 255.85714285714272), (522.085714285714, 258.00000000000006), (695.2285714285713, 350.8571428571427), (600.142857142857, 338.9999999999998), (552.9428571428572, 351.1142857142855), (492.142857142857, 349.85714285714266), (709.9999999999999, 382.99999999999983), (631.1142857142853, 398.9999999999999), (570.142857142857, 413.9999999999997), (521.9428571428568, 413.85714285714255), (724.9428571428571, 429.9999999999999), (662.1428571428568, 443.85714285714266), (633.2857142857141, 476.9999999999997), (602.2857142857139, 492.08571428571435)], [(849.9999999999995, 333.8571428571427), (809.571428571428, 304.9999999999999), (772.6285714285707, 263.5714285714284), (724.8571428571424, 241.99999999999986), (660.8571428571425, 222.14285714285703), (700.0571428571424, 304.7428571428571), (616.428571428571, 274.4285714285713), (568.8571428571425, 263.5714285714284), (521.8571428571425, 257.9999999999999), (691.885714285714, 351.3714285714284), (601.4285714285711, 342.19999999999976), (553.0285714285714, 351.7428571428568), (489.0285714285711, 357.57142857142827), (709.9999999999998, 382.99999999999966), (631.7428571428566, 398.9999999999999), (567.028571428571, 413.99999999999966), (522.028571428571, 414.37142857142834), (725.028571428571, 429.99999999999966), (663.4285714285709, 451.5714285714283), (629.8571428571425, 476.9999999999997), (598.8571428571425, 491.8571428571428)], [(849.9999999999995, 334.71428571428555), (815.1428571428567, 304.99999999999983), (776.2571428571423, 269.142857142857), (724.9142857142854, 241.99999999999986), (661.7142857142853, 227.28571428571416), (705.3142857142852, 305.0857142857141), (619.8571428571425, 277.85714285714266), (568.9142857142854, 269.142857142857), (521.9142857142854, 257.9999999999999), (692.9714285714282, 351.74285714285696), (604.8571428571425, 345.39999999999975), (553.2571428571425, 352.0857142857141), (490.05714285714254, 363.14285714285694), (709.9999999999997, 382.9999999999998), (632.085714285714, 398.9999999999998), (568.0571428571425, 413.9999999999998), (522.2571428571425, 414.7428571428569), (725.2571428571425, 429.9999999999998), (666.8571428571424, 457.1428571428569), (630.7142857142853, 476.9999999999997), (599.7142857142853, 491.91428571428537)], [(849.9999999999995, 339.85714285714266), (820.2857142857138, 304.99999999999983), (783.7428571428567, 274.2857142857141), (723.9714285714282, 241.99999999999986), (666.8571428571424, 225.9999999999999), (710.4571428571425, 304.99999999999983), (627.0571428571425, 285.142857142857), (569.2571428571425, 274.2857142857141), (522.2571428571425, 257.9999999999999), (698.5999999999996, 352.0857142857141), (612.1428571428569, 346.5428571428569), (553.7428571428568, 351.99999999999983), (495.2571428571425, 368.2857142857141), (708.7142857142853, 382.9999999999998), (631.9999999999997, 398.9999999999998), (573.2571428571425, 413.9999999999998), (521.4571428571426, 415.08571428571406), (725.7428571428568, 429.9999999999998), (674.1428571428568, 462.28571428571405), (635.8571428571424, 476.9999999999997), (604.8571428571425, 492.2571428571425)], [(849.9999999999995, 347.14285714285694), (818.9999999999995, 304.99999999999983), (788.9428571428567, 272.99999999999983), (729.5999999999996, 241.99999999999986), (674.1428571428568, 225.9999999999999), (709.7428571428568, 304.99999999999983), (632.5428571428567, 290.2857142857141), (568.4571428571425, 272.99999999999983), (522.7428571428568, 257.9999999999999), (706.2285714285711, 351.99999999999983), (617.285714285714, 352.6857142857141), (554.085714285714, 351.99999999999983), (502.74285714285685, 366.9999999999998), (713.8571428571424, 382.9999999999998), (631.9999999999997, 398.9999999999998), (580.7428571428568, 413.9999999999998), (526.9428571428568, 414.9999999999998), (724.7999999999995, 429.9999999999998), (679.1999999999996, 460.9999999999998), (643.1428571428568, 476.9999999999997), (612.1428571428569, 492.74285714285685)], [(849.9142857142851, 352.2857142857141), (818.9999999999995, 304.99999999999983), (788.0857142857137, 272.99999999999983), (737.2285714285711, 241.99999999999986), (679.1999999999996, 225.9999999999999), (710.085714285714, 304.99999999999983), (631.7428571428568, 288.9999999999999), (573.9428571428568, 272.99999999999983), (521.7999999999997, 257.9999999999999), (711.2857142857139, 351.99999999999983), (615.9999999999997, 351.25714285714264), (553.9999999999998, 351.99999999999983), (507.9428571428569, 366.9999999999998), (721.1428571428567, 382.9999999999998), (630.7142857142853, 398.9999999999998), (585.9428571428568, 413.9999999999998), (534.1428571428568, 414.9999999999998), (729.8571428571424, 429.9999999999998), (678.2571428571425, 460.9999999999998), (648.2857142857139, 476.9999999999997), (617.285714285714, 493.085714285714)], [(850.2571428571423, 350.9999999999998), (818.9999999999995, 304.99999999999983), (787.9999999999995, 272.99999999999983), (742.2857142857138, 241.99999999999986), (678.2571428571425, 225.9999999999999), (709.9999999999997, 304.99999999999983), (632.085714285714, 288.9999999999999), (581.1428571428568, 272.99999999999983), (526.8571428571424, 257.9999999999999), (709.9999999999997, 351.99999999999983), (615.9999999999997, 351.17142857142835), (552.7142857142854, 351.99999999999983), (507.085714285714, 366.9999999999998), (726.2857142857139, 381.7142857142855), (635.8571428571424, 398.9999999999998), (585.085714285714, 413.9999999999998), (539.285714285714, 413.7142857142855), (737.1428571428567, 429.9999999999998), (678.7428571428568, 460.9999999999998), (646.9142857142854, 476.9999999999997), (615.9999999999997, 492.9999999999997)], [(849.5428571428566, 350.9999999999998), (818.9999999999995, 304.99999999999983), (787.9999999999995, 272.99999999999983), (740.9999999999995, 241.99999999999986), (678.7428571428568, 225.9999999999999), (708.7142857142853, 304.99999999999983), (631.9999999999997, 288.9999999999999), (586.2857142857139, 272.99999999999983), (534.1428571428568, 257.9999999999999), (709.9999999999997, 351.99999999999983), (614.7142857142853, 351.74285714285696), (557.8571428571425, 351.99999999999983), (506.9999999999997, 366.9999999999998), (724.9999999999995, 386.85714285714266), (643.1428571428568, 398.9999999999998), (584.9999999999997, 413.9999999999998), (537.9999999999997, 418.8571428571426), (742.2857142857138, 429.9999999999998), (677.7999999999996, 460.9999999999998), (647.2571428571425, 476.9999999999997), (615.9999999999997, 492.9999999999997)], [(854.5999999999995, 350.9999999999998), (817.7142857142852, 304.99999999999983), (786.7142857142852, 272.99999999999983), (740.9999999999995, 241.91428571428557), (679.085714285714, 225.9999999999999), (713.8571428571424, 304.99999999999983), (631.9999999999997, 288.9999999999999), (584.9999999999997, 272.99999999999983), (539.285714285714, 257.9999999999999), (709.9999999999997, 351.99999999999983), (619.8571428571425, 352.0857142857141), (565.1428571428569, 351.99999999999983), (505.71428571428544, 366.9999999999998), (724.9999999999995, 394.14285714285694), (648.2857142857139, 397.71428571428544), (584.9999999999997, 413.9999999999998), (537.9999999999997, 426.1428571428569), (740.9999999999995, 429.9999999999998), (682.8571428571424, 460.9999999999998), (647.7428571428568, 476.9999999999997), (614.7142857142853, 492.9999999999997)], [(861.6571428571424, 350.9999999999998), (822.8571428571424, 304.99999999999983), (791.8571428571424, 272.99999999999983), (740.9999999999995, 242.2571428571427), (677.7142857142853, 225.9142857142856), (721.1428571428567, 304.99999999999983), (630.7142857142853, 288.9999999999999), (584.9999999999997, 272.99999999999983), (537.9999999999997, 257.9999999999999), (708.7142857142853, 351.99999999999983), (627.0571428571425, 351.99999999999983), (570.285714285714, 351.99999999999983), (510.85714285714255, 366.9999999999998), (724.9999999999995, 399.2857142857141), (646.9999999999995, 402.85714285714266), (583.7142857142853, 413.9999999999998), (537.9999999999997, 431.28571428571405), (740.9999999999995, 429.9999999999998), (690.1428571428568, 460.9999999999998), (646.8857142857139, 476.9999999999997), (619.8571428571425, 492.9999999999997)], [(866.9428571428566, 350.9999999999998), (830.0571428571425, 304.99999999999983), (799.1428571428567, 272.99999999999983), (740.9999999999995, 242.82857142857125), (682.8571428571424, 226.34285714285699), (726.2857142857139, 304.99999999999983), (635.8571428571424, 288.9999999999999), (584.9999999999997, 272.99999999999983), (537.9999999999997, 257.9999999999999), (715.1428571428568, 351.99999999999983), (632.5428571428567, 351.99999999999983), (568.9999999999997, 351.99999999999983), (518.1428571428569, 366.9999999999998), (724.9142857142854, 399.2857142857141), (646.9142857142854, 411.4285714285712), (588.8571428571424, 413.9999999999998), (537.9999999999997, 431.28571428571405), (740.9999999999995, 429.9999999999998), (695.2857142857139, 460.9999999999998), (651.5142857142853, 476.9999999999997), (627.1428571428569, 493.085714285714)], [(867.3714285714279, 353.5714285714283), (835.5428571428567, 306.3714285714284), (804.2857142857138, 274.2857142857141), (740.9999999999995, 244.11428571428556), (691.4285714285711, 227.77142857142846), (726.2857142857139, 306.3714285714284), (644.4285714285711, 290.3714285714284), (584.9999999999997, 274.2857142857141), (537.9999999999997, 259.3714285714284), (717.2857142857139, 353.3714285714284), (633.1142857142854, 353.3714285714284), (568.9999999999997, 353.2857142857141), (524.5714285714282, 368.2857142857141), (725.3428571428567, 394.14285714285694), (647.3428571428567, 412.7999999999998), (597.4285714285711, 415.28571428571405), (537.9999999999997, 426.2285714285712), (740.9999999999995, 431.28571428571405), (693.9999999999995, 462.37142857142834), (659.9428571428567, 478.3714285714283), (633.5714285714282, 494.0285714285711)], [(862.2285714285708, 344.6571428571426), (836.1142857142853, 302.1714285714284), (804.2857142857138, 270.51428571428556), (742.3714285714282, 239.77142857142843), (692.7999999999996, 223.85714285714272), (721.2285714285711, 302.1714285714284), (644.5142857142853, 286.1714285714284), (586.3714285714282, 270.51428571428556), (539.3714285714282, 255.17142857142844), (716.5142857142853, 349.25714285714264), (627.9714285714282, 349.25714285714264), (570.285714285714, 348.22857142857123), (519.5142857142855, 364.5142857142855), (726.7714285714281, 388.22857142857123), (648.7714285714283, 400.0285714285712), (598.7142857142853, 411.5142857142855), (539.3714285714282, 419.9714285714283), (742.2857142857138, 427.51428571428545), (695.3714285714282, 458.1714285714283), (660.1714285714282, 474.1714285714283), (628.4285714285711, 489.68571428571397)], [(855.8857142857137, 323.3142857142856), (832.3428571428566, 289.2571428571427), (800.5142857142853, 257.742857142857), (736.8857142857139, 226.65714285714273), (678.7428571428568, 210.65714285714273), (714.8857142857139, 289.2571428571427), (636.8857142857139, 273.2571428571427), (580.8857142857139, 257.742857142857), (535.1714285714282, 242.2571428571427), (704.5999999999996, 335.9142857142855), (621.3999999999996, 335.9999999999998), (566.5142857142854, 341.8857142857141), (506.7428571428569, 351.74285714285696), (721.5714285714282, 377.5999999999998), (644.8571428571424, 380.6857142857141), (585.085714285714, 398.7428571428569), (535.1714285714282, 408.85714285714266), (738.5142857142853, 414.7428571428569), (691.1714285714281, 445.25714285714264), (652.9714285714282, 461.2571428571425), (616.085714285714, 476.9999999999997)], [(845.1142857142852, 310.3142857142856), (818.9142857142853, 275.1999999999998), (787.7428571428568, 243.4571428571427), (729.1142857142853, 212.542857142857), (665.8285714285711, 197.8285714285713), (704.1142857142852, 275.1999999999998), (626.1142857142853, 260.48571428571415), (574.3999999999996, 244.8285714285713), (522.2571428571425, 228.19999999999987), (698.1142857142853, 321.71428571428555), (610.8571428571425, 322.6571428571427), (553.7428571428568, 329.82857142857125), (495.1142857142854, 338.82857142857125), (713.5142857142853, 371.02857142857124), (631.6571428571425, 374.3428571428569), (572.3142857142853, 384.542857142857), (522.2571428571425, 401.94285714285695), (724.4571428571425, 400.54285714285686), (678.2571428571425, 431.1999999999997), (642.1999999999996, 447.1999999999997), (603.3142857142855, 462.9428571428569)], [(837.4857142857138, 313.74285714285696), (804.7142857142852, 275.1999999999998), (774.828571428571, 244.74285714285702), (723.628571428571, 212.1142857142856), (661.628571428571, 192.25714285714272), (697.7714285714281, 276.57142857142844), (619.7714285714283, 255.42857142857127), (562.4857142857139, 239.25714285714272), (509.48571428571404, 229.57142857142844), (692.628571428571, 323.2285714285713), (604.5142857142854, 317.5999999999998), (540.8285714285711, 328.62857142857126), (484.485714285714, 333.34285714285693), (709.1714285714281, 364.5999999999998), (618.7428571428568, 376.5714285714284), (567.9714285714282, 385.39999999999975), (509.48571428571404, 395.59999999999974), (716.6857142857139, 400.1142857142855), (665.4857142857139, 432.485714285714), (635.428571428571, 448.48571428571404), (598.9714285714282, 464.085714285714)], [(838.8571428571424, 325.88571428571413), (807.5142857142852, 289.2571428571427), (770.5428571428567, 253.88571428571416), (724.9999999999995, 226.25714285714272), (662.9999999999995, 199.1142857142856), (691.428571428571, 283.85714285714266), (614.7142857142853, 261.85714285714266), (561.7142857142854, 246.1142857142856), (504.4285714285711, 238.142857142857), (693.9999999999995, 330.94285714285695), (599.2285714285711, 324.5999999999998), (535.3428571428568, 330.85714285714266), (483.71428571428544, 339.7714285714284), (704.028571428571, 371.5999999999998), (613.685714285714, 388.05714285714265), (568.9142857142854, 393.11428571428553), (504.4285714285711, 402.5999999999998), (719.7714285714281, 414.25714285714264), (661.7142857142853, 441.39999999999975), (630.2285714285711, 456.1142857142854), (599.828571428571, 471.51428571428545)], [(840.9999999999995, 332.22857142857123), (810.0857142857138, 302.2571428571427), (771.0571428571425, 259.3714285714284), (724.9142857142854, 239.17142857142844), (662.9999999999995, 206.8857142857142), (697.8571428571424, 295.4285714285713), (615.9999999999997, 269.142857142857), (563.8571428571425, 253.88571428571416), (510.85714285714255, 243.28571428571416), (693.9999999999995, 342.4285714285712), (600.6571428571425, 332.22857142857123), (541.8571428571425, 337.2857142857141), (485.85714285714255, 347.3999999999998), (702.4571428571425, 379.22857142857123), (620.2571428571425, 394.4857142857141), (568.9999999999997, 405.17142857142835), (510.85714285714255, 410.22857142857123), (726.2857142857139, 428.45714285714257), (661.7142857142853, 447.0285714285712), (631.6571428571425, 468.2571428571426), (600.2571428571425, 483.51428571428545)], [(845.2857142857138, 337.2857142857141), (812.9999999999994, 307.39999999999986), (776.3428571428567, 258.0857142857141), (725.3428571428567, 244.65714285714273), (662.9999999999995, 212.4571428571427), (705.1428571428568, 296.2857142857141), (615.9999999999997, 275.57142857142844), (570.285714285714, 259.4571428571427), (518.1428571428569, 241.99999999999986), (693.9142857142854, 343.2857142857141), (601.085714285714, 337.2857142857141), (549.1428571428569, 335.9999999999998), (492.285714285714, 353.11428571428553), (704.9428571428567, 385.5714285714283), (627.7428571428568, 401.05714285714265), (568.9999999999997, 406.37142857142834), (518.1428571428569, 416.5714285714283), (724.9999999999995, 427.59999999999974), (666.8571428571424, 446.1714285714283), (632.085714285714, 469.1142857142854), (600.7428571428568, 484.285714285714)], [(839.7142857142853, 337.2857142857141), (813.8571428571424, 300.3999999999998), (783.3999999999995, 257.742857142857), (725.3999999999996, 238.22857142857129), (661.7142857142853, 210.74285714285702), (710.2857142857139, 294.142857142857), (614.7142857142853, 269.142857142857), (568.9999999999997, 257.742857142857), (523.285714285714, 241.99999999999986), (694.3428571428567, 341.14285714285694), (599.7142857142853, 335.9999999999998), (554.1999999999996, 335.9999999999998), (490.9999999999998, 351.82857142857125), (711.2857142857139, 379.14285714285694), (632.9428571428568, 394.4857142857141), (568.9142857142854, 404.1428571428569), (523.1999999999997, 410.14285714285694), (724.9142857142854, 418.59999999999974), (674.1428571428568, 445.7428571428569), (630.7142857142853, 466.39999999999975), (599.7999999999997, 482.1428571428569)], [(842.6285714285709, 332.14285714285694), (818.9999999999995, 292.77142857142843), (788.4571428571425, 257.17142857142846), (724.3142857142852, 230.51428571428556), (666.7714285714283, 210.17142857142846), (708.9142857142853, 287.71428571428555), (619.7714285714283, 260.57142857142844), (567.6285714285711, 257.17142857142846), (520.6285714285711, 241.99999999999986), (693.1142857142853, 334.71428571428555), (604.8571428571425, 335.9999999999998), (551.9714285714283, 335.9142857142855), (489.62857142857115, 351.17142857142835), (708.7142857142853, 370.5714285714283), (630.7999999999996, 385.48571428571404), (567.9714285714282, 397.71428571428544), (520.9714285714283, 401.5714285714283), (723.9714285714282, 412.97142857142836), (677.9142857142854, 445.17142857142835), (635.7714285714281, 459.62857142857115), (604.8571428571425, 475.71428571428544)], [(848.9714285714281, 323.5714285714284), (822.7714285714281, 286.4285714285713), (786.4571428571425, 257.17142857142846), (729.5142857142853, 223.942857142857), (673.1142857142854, 210.17142857142846), (707.9714285714281, 287.71428571428555), (626.1142857142853, 260.48571428571415), (573.1142857142854, 257.17142857142846), (526.1142857142854, 240.6285714285713), (697.1714285714281, 335.9999999999998), (610.7714285714283, 335.9999999999998), (557.5142857142854, 335.0571428571427), (493.82857142857114, 351.17142857142835), (713.8571428571424, 370.5714285714283), (635.8571428571424, 385.48571428571404), (572.3142857142853, 397.71428571428544), (525.3142857142855, 401.5714285714283), (729.5999999999996, 413.8285714285712), (682.1142857142854, 445.17142857142835), (642.1142857142853, 459.71428571428544), (610.7714285714283, 475.71428571428544)], [(863.171428571428, 323.5714285714284), (830.3999999999995, 292.77142857142843), (791.8571428571424, 256.4571428571427), (735.8571428571424, 230.51428571428556), (683.8857142857139, 209.45714285714274), (713.5142857142853, 292.85714285714266), (636.8857142857139, 269.39999999999986), (579.5999999999997, 257.742857142857), (532.5999999999997, 246.1142857142856), (709.6571428571425, 334.71428571428555), (621.3999999999996, 335.9999999999998), (564.1999999999997, 340.34285714285693), (506.6571428571426, 351.74285714285696), (719.7714285714281, 379.14285714285694), (641.8571428571424, 394.3999999999998), (584.9999999999997, 402.85714285714266), (537.9999999999997, 410.14285714285694), (735.9428571428567, 412.97142857142836), (689.8857142857139, 445.7428571428569), (652.8857142857139, 464.77142857142826), (621.3999999999996, 480.8571428571426)], [(867.3714285714279, 332.14285714285694), (836.028571428571, 300.3999999999998), (799.3999999999996, 261.94285714285695), (746.3999999999995, 238.22857142857129), (690.1428571428568, 214.94285714285704), (720.1999999999996, 300.142857142857), (643.1428571428568, 275.0285714285713), (590.1428571428568, 258.0857142857141), (543.1428571428569, 253.88571428571416), (722.428571428571, 339.85714285714266), (627.8857142857139, 335.9999999999998), (574.8857142857139, 347.3999999999998), (519.7714285714283, 350.79999999999984), (730.3999999999996, 384.28571428571405), (652.0571428571425, 400.0285714285712), (597.685714285714, 410.14285714285694), (550.6857142857139, 415.19999999999976), (746.1428571428567, 418.59999999999974), (695.2857142857139, 444.7999999999997), (659.2285714285711, 472.39999999999975), (627.8857142857139, 488.1428571428569)], [(865.9999999999994, 337.2857142857141), (835.0857142857138, 306.02857142857124), (805.028571428571, 270.4285714285713), (752.8857142857139, 243.28571428571416), (695.5428571428567, 222.14285714285703), (730.8857142857139, 305.2857142857141), (648.5428571428567, 274.0857142857141), (596.3999999999996, 257.9999999999999), (549.3999999999996, 259.3714285714284), (726.5428571428567, 347.14285714285694), (633.3714285714282, 335.9999999999998), (581.2285714285711, 352.542857142857), (524.0285714285711, 357.142857142857), (736.8857142857139, 382.9999999999998), (658.3999999999996, 399.08571428571406), (602.028571428571, 415.28571428571405), (555.0285714285711, 414.34285714285693), (752.0571428571424, 426.2285714285712), (693.0571428571425, 451.1428571428569), (664.2857142857139, 478.0285714285711), (633.3714285714282, 493.285714285714)], [(865.9999999999994, 335.9999999999998), (834.9999999999995, 305.1714285714284), (804.0857142857138, 270.4285714285713), (758.3714285714282, 241.99999999999986), (694.7428571428568, 227.28571428571416), (738.5142857142853, 303.99999999999983), (647.7428571428568, 274.0857142857141), (602.028571428571, 257.9999999999999), (555.0285714285711, 257.9999999999999), (725.828571428571, 353.5714285714283), (631.9999999999997, 335.9999999999998), (586.2857142857139, 351.4857142857141), (523.085714285714, 359.2857142857141), (742.3714285714282, 382.9999999999998), (664.028571428571, 399.08571428571406), (601.085714285714, 413.9999999999998), (554.085714285714, 414.48571428571404), (757.628571428571, 432.5714285714283), (699.628571428571, 453.285714285714), (662.9999999999995, 477.085714285714), (631.9999999999997, 491.99999999999966)], [(865.9999999999994, 335.9999999999998), (834.9999999999995, 304.74285714285696), (803.9999999999995, 261.85714285714266), (757.085714285714, 241.99999999999986), (695.1714285714282, 227.28571428571416), (738.428571428571, 305.2857142857141), (648.1714285714281, 273.742857142857), (601.1714285714282, 257.9999999999999), (554.1714285714284, 258.0857142857141), (725.828571428571, 347.14285714285694), (631.9999999999997, 335.9999999999998), (585.085714285714, 352.62857142857126), (522.9999999999997, 357.142857142857), (741.0857142857138, 382.9999999999998), (664.3714285714282, 398.7428571428569), (600.9999999999997, 415.28571428571405), (553.9999999999998, 414.34285714285693), (756.4857142857139, 426.2285714285712), (701.628571428571, 451.1428571428569), (662.9999999999995, 478.285714285714), (631.9999999999997, 493.285714285714)], [(867.2857142857138, 335.9999999999998), (835.0857142857138, 305.542857142857), (804.0857142857138, 256.71428571428555), (756.7428571428568, 243.28571428571416), (694.7428571428568, 222.14285714285703), (729.9428571428567, 300.142857142857), (647.7428571428568, 273.2571428571427), (600.7428571428568, 257.9999999999999), (555.0285714285711, 259.0285714285713), (725.2571428571425, 339.85714285714266), (631.9999999999997, 335.9999999999998), (585.9428571428568, 347.05714285714265), (523.085714285714, 350.71428571428555), (741.9428571428567, 384.28571428571405), (659.1428571428568, 399.5428571428569), (601.085714285714, 410.14285714285694), (554.085714285714, 413.9142857142855), (757.628571428571, 418.59999999999974), (699.0571428571425, 444.7999999999997), (662.9999999999995, 473.2285714285712), (631.9999999999997, 488.1428571428569)], [(862.1428571428567, 335.9999999999998), (834.7428571428567, 300.0571428571427), (803.7428571428567, 257.9999999999999), (756.2571428571424, 238.142857142857), (694.2571428571425, 214.85714285714275), (724.4571428571425, 292.85714285714266), (647.2571428571425, 274.1999999999998), (601.5428571428567, 258.0857142857141), (549.3999999999996, 253.39999999999986), (726.1999999999996, 334.71428571428555), (632.085714285714, 335.9999999999998), (580.7428571428568, 339.85714285714266), (522.7428571428568, 352.0857142857141), (736.7428571428568, 379.14285714285694), (651.9428571428568, 394.05714285714265), (602.028571428571, 402.94285714285695), (553.7428571428568, 415.28571428571405), (752.0571428571424, 412.97142857142836), (692.7142857142853, 445.7428571428569), (662.9999999999995, 465.59999999999974), (631.9999999999997, 480.8571428571426)], [(854.9428571428566, 335.9999999999998), (834.2571428571424, 292.85714285714266), (803.2571428571424, 258.0857142857141), (755.9142857142853, 230.85714285714272), (693.9142857142854, 209.71428571428558), (725.2571428571425, 287.71428571428555), (646.9142857142854, 269.142857142857), (596.0571428571425, 259.0285714285713), (541.7714285714283, 245.77142857142843), (721.1428571428567, 335.9999999999998), (631.7428571428568, 337.2857142857141), (573.2571428571425, 334.71428571428555), (522.2571428571425, 353.02857142857124), (729.2571428571424, 371.94285714285695), (646.4571428571426, 386.85714285714266), (595.1142857142854, 397.45714285714257), (553.2571428571425, 410.14285714285694), (744.8571428571423, 413.9142857142855), (693.9999999999995, 446.5428571428569), (663.085714285714, 459.9714285714283), (631.9999999999997, 475.71428571428544)], [(849.371428571428, 337.2857142857141), (833.9142857142853, 287.71428571428555), (802.9142857142853, 257.742857142857), (755.9999999999995, 225.7142857142856), (693.9999999999995, 210.99999999999986), (724.9142857142854, 288.9999999999999), (646.9999999999995, 261.85714285714266), (588.8571428571424, 253.39999999999986), (536.7142857142853, 240.71428571428558), (712.5714285714282, 335.9999999999998), (631.2571428571425, 332.14285714285694), (568.0571428571425, 335.9999999999998), (521.9142857142854, 347.3999999999998), (724.0571428571425, 366.4571428571426), (647.2571428571425, 381.7142857142855), (592.628571428571, 398.25714285714264), (552.9142857142854, 402.85714285714266), (739.7142857142852, 413.9999999999998), (693.9999999999995, 441.0571428571426), (662.6571428571425, 462.19999999999976), (631.9999999999997, 478.285714285714)], [(850.5999999999995, 332.22857142857123), (833.9999999999995, 288.9999999999999), (802.9999999999995, 257.2571428571427), (755.9999999999995, 227.08571428571418), (693.9999999999995, 210.99999999999986), (724.9999999999995, 290.2857142857141), (646.9999999999995, 256.71428571428555), (583.7142857142853, 245.77142857142843), (537.9999999999997, 243.28571428571416), (713.8571428571424, 337.2857142857141), (630.8285714285711, 324.94285714285695), (568.9142857142854, 337.2857142857141), (521.9999999999997, 339.7714285714284), (724.9142857142854, 367.25714285714264), (646.8285714285711, 382.9999999999998), (593.4857142857139, 399.1999999999998), (551.6285714285711, 397.71428571428544), (740.9999999999995, 415.28571428571405), (693.9999999999995, 433.85714285714255), (662.5142857142853, 457.1428571428569), (630.7142857142853, 473.2285714285712)], [(850.3999999999994, 324.5999999999998), (835.2857142857139, 290.2857142857141), (802.9999999999995, 258.1999999999998), (755.9999999999995, 226.74285714285702), (693.9999999999995, 210.99999999999986), (726.2857142857139, 285.2285714285713), (648.2857142857139, 257.9999999999999), (584.9999999999997, 240.71428571428558), (537.9999999999997, 238.142857142857), (717.2857142857139, 332.22857142857123), (631.3428571428568, 319.4571428571427), (568.9999999999997, 332.14285714285694), (523.285714285714, 334.71428571428555), (724.9999999999995, 366.9142857142855), (645.9714285714282, 384.37142857142834), (605.3999999999996, 394.14285714285694), (555.7428571428568, 398.9999999999998), (740.9999999999995, 410.14285714285694), (693.9999999999995, 429.9999999999998), (661.3714285714282, 449.9428571428569), (634.5714285714282, 465.59999999999974)], [(850.3428571428567, 318.9714285714284), (830.1428571428568, 285.142857142857), (802.9999999999995, 253.142857142857), (755.9999999999995, 226.25714285714272), (693.9999999999995, 210.99999999999986), (721.1428571428567, 277.59999999999985), (643.1428571428568, 258.0857142857141), (584.9999999999997, 241.99999999999986), (537.9999999999997, 230.942857142857), (715.1428571428568, 324.5999999999998), (632.7714285714283, 320.2571428571427), (568.9999999999997, 324.85714285714266), (518.1428571428569, 335.9999999999998), (726.2857142857139, 368.2857142857141), (652.9714285714282, 377.5142857142855), (610.5142857142854, 386.85714285714266), (563.685714285714, 400.37142857142834), (742.2857142857138, 402.85714285714266), (693.9999999999995, 424.85714285714255), (665.5714285714282, 445.7428571428569), (642.8857142857139, 463.9999999999997)], [(849.9142857142851, 321.1999999999998), (822.8571428571424, 277.85714285714266), (802.9999999999995, 245.77142857142843), (755.9999999999995, 225.8285714285713), (692.628571428571, 210.99999999999986), (713.8571428571424, 271.9714285714284), (635.8571428571424, 259.0285714285713), (584.9999999999997, 241.99999999999986), (537.9999999999997, 225.4571428571427), (708.7142857142853, 318.9714285714284), (627.4857142857139, 319.91428571428554), (568.9999999999997, 319.71428571428555), (510.85714285714255, 335.9999999999998), (721.1428571428567, 361.85714285714266), (653.7428571428568, 376.5142857142855), (618.8285714285711, 382.9999999999998), (598.1142857142854, 396.25714285714264), (735.8571428571424, 397.71428571428544), (692.628571428571, 424.08571428571406), (676.9714285714282, 442.7714285714283), (673.1428571428567, 448.82857142857114)], [(851.2857142857138, 314.85714285714266), (817.7142857142852, 271.4285714285713), (802.9999999999995, 239.68571428571414), (755.9142857142853, 224.97142857142842), (698.1142857142853, 209.6285714285713), (708.7142857142853, 272.91428571428554), (630.7142857142853, 253.39999999999986), (584.9999999999997, 243.28571428571416), (537.9999999999997, 226.25714285714272), (709.9999999999997, 319.91428571428554), (619.7714285714283, 319.99999999999983), (568.9999999999997, 321.0857142857141), (505.71428571428544, 335.9999999999998), (713.8571428571424, 359.71428571428555), (656.5142857142853, 373.6571428571426), (635.8571428571424, 379.22857142857123), (632.6857142857139, 382.9999999999998), (733.7999999999995, 398.9999999999998), (698.1142857142853, 420.7428571428569), (693.6571428571424, 430.9428571428569), (690.7999999999996, 427.45714285714257)], [(844.8571428571424, 313.99999999999983), (818.9999999999995, 277.85714285714266), (802.9142857142853, 246.59999999999985), (754.9714285714282, 229.31428571428557), (703.2285714285711, 213.8285714285713), (709.9999999999997, 272.99999999999983), (631.9999999999997, 245.77142857142843), (584.9999999999997, 238.142857142857), (537.9999999999997, 227.19999999999987), (709.9999999999997, 321.2857142857141), (614.7142857142853, 321.2857142857141), (568.9999999999997, 320.742857142857), (506.9999999999997, 337.2857142857141), (707.4285714285711, 363.14285714285694), (657.5142857142853, 373.31428571428546), (651.085714285714, 372.885714285714), (643.7142857142853, 372.39999999999975), (735.5142857142853, 400.37142857142834), (704.5999999999996, 420.685714285714), (707.9714285714281, 414.34285714285693), (711.6571428571424, 409.7999999999998)], [(842.7142857142852, 310.99999999999983), (817.7142857142852, 285.142857142857), (803.2571428571424, 252.942857142857), (761.8857142857139, 240.71428571428558), (720.7142857142853, 223.99999999999986), (709.9999999999997, 272.99999999999983), (631.9999999999997, 240.71428571428558), (584.9999999999997, 230.942857142857), (537.9999999999997, 222.14285714285703), (709.9999999999997, 314.85714285714266), (615.9999999999997, 316.142857142857), (568.9999999999997, 320.2571428571427), (506.9999999999997, 332.14285714285694), (713.8571428571424, 364.4285714285712), (663.085714285714, 366.4571428571426), (681.5999999999997, 362.11428571428553), (676.1142857142853, 357.6571428571427), (740.5142857142853, 394.8857142857141), (715.1428571428568, 408.7714285714284), (730.0571428571425, 397.3714285714283), (735.428571428571, 390.28571428571405)], [(844.8571428571424, 310.142857142857), (824.1428571428568, 290.2857142857141), (802.4571428571423, 261.77142857142843), (764.3714285714282, 255.88571428571413), (727.5428571428567, 243.79999999999984), (709.9999999999997, 272.99999999999983), (631.9999999999997, 241.99999999999986), (584.9999999999997, 225.4571428571427), (537.9999999999997, 214.85714285714275), (709.9999999999997, 312.71428571428555), (615.9999999999997, 308.85714285714266), (568.9999999999997, 319.91428571428554), (505.71428571428544, 324.85714285714266), (721.1428571428567, 355.85714285714266), (666.7714285714283, 367.25714285714264), (707.6857142857139, 355.7714285714284), (727.6857142857139, 343.97142857142836), (744.5142857142853, 387.11428571428553), (720.1999999999996, 402.85714285714266), (748.3714285714282, 387.11428571428553), (767.028571428571, 362.4571428571426)], [(851.2857142857138, 303.79999999999984), (824.9142857142853, 288.9999999999999), (807.9428571428566, 273.2571428571427), (760.8571428571424, 278.0571428571427), (737.1714285714281, 278.79999999999984), (709.9999999999997, 272.99999999999983), (631.9999999999997, 241.99999999999986), (584.9999999999997, 226.25714285714272), (537.9999999999997, 209.71428571428558), (709.9999999999997, 316.142857142857), (615.9999999999997, 303.71428571428555), (568.9999999999997, 319.99999999999983), (510.85714285714255, 319.79999999999984), (726.1999999999996, 350.79999999999984), (674.3999999999996, 366.9142857142855), (734.5142857142853, 350.79999999999984), (771.4857142857139, 333.34285714285693), (753.5142857142853, 381.6285714285712), (731.828571428571, 397.7999999999997), (764.628571428571, 376.4857142857141), (779.2571428571424, 348.142857142857)], [(849.9999999999995, 304.74285714285696), (828.3428571428567, 288.9999999999999), (816.428571428571, 286.1714285714284), (759.5714285714282, 297.65714285714273), (731.6571428571424, 309.88571428571413), (709.9999999999997, 272.99999999999983), (631.9999999999997, 241.99999999999986), (584.9999999999997, 225.9142857142856), (537.9999999999997, 210.99999999999986), (709.9999999999997, 317.4285714285712), (615.9999999999997, 304.99999999999983), (568.9999999999997, 319.99999999999983), (519.4285714285711, 320.742857142857), (725.3428571428567, 351.74285714285696), (680.1142857142853, 366.9999999999998), (737.9142857142853, 351.6571428571427), (765.6571428571425, 335.9999999999998), (753.4285714285709, 382.9999999999998), (732.8857142857139, 398.7428571428569), (764.1142857142853, 375.71428571428544), (784.228571428571, 351.6571428571427)], [(849.9999999999995, 304.25714285714264), (829.3428571428567, 288.9999999999999), (816.428571428571, 290.3714285714284), (769.428571428571, 306.11428571428553), (731.3142857142852, 327.2571428571427), (709.9999999999997, 272.99999999999983), (631.9999999999997, 241.99999999999986), (584.9999999999997, 225.9142857142856), (537.9999999999997, 210.99999999999986), (709.9999999999997, 308.85714285714266), (615.9999999999997, 304.99999999999983), (568.9999999999997, 319.99999999999983), (519.4285714285711, 320.1714285714284), (725.4857142857138, 351.17142857142835), (678.828571428571, 366.9142857142855), (737.7714285714281, 351.5142857142855), (758.028571428571, 335.9142857142855), (744.8571428571423, 382.9999999999998), (730.8857142857139, 398.17142857142835), (762.6857142857139, 377.85714285714266), (790.6571428571424, 350.22857142857123)], [(849.9999999999995, 303.91428571428554), (835.628571428571, 287.71428571428555), (807.8571428571423, 288.9999999999999), (769.428571428571, 308.85714285714266), (725.7428571428568, 335.9999999999998), (709.9999999999997, 272.91428571428554), (631.9999999999997, 241.91428571428557), (584.9999999999997, 226.25714285714272), (539.285714285714, 209.71428571428558), (709.9999999999997, 302.4285714285713), (615.9999999999997, 303.71428571428555), (568.9999999999997, 319.99999999999983), (510.85714285714255, 320.1714285714284), (726.628571428571, 351.17142857142835), (678.1714285714281, 367.25714285714264), (730.0571428571425, 350.3714285714284), (745.1142857142852, 334.9714285714284), (739.7142857142852, 382.9999999999998), (724.5428571428567, 398.17142857142835), (750.771428571428, 382.9999999999998), (786.5428571428566, 354.14285714285694)], [(849.9999999999995, 303.91428571428554), (833.9142857142853, 292.77142857142843), (802.7142857142852, 287.71428571428555), (760.8571428571424, 315.11428571428553), (721.3999999999996, 347.3999999999998), (709.9999999999997, 271.9714285714284), (631.9999999999997, 241.0571428571427), (584.9999999999997, 226.74285714285702), (532.8571428571424, 214.85714285714275), (709.9999999999997, 308.85714285714266), (615.9999999999997, 308.85714285714266), (568.9999999999997, 319.99999999999983), (505.71428571428544, 320.742857142857), (721.0571428571425, 351.82857142857125), (678.2571428571425, 366.4571428571426), (719.8571428571424, 355.94285714285695), (735.7714285714281, 339.3142857142855), (740.9999999999995, 382.9999999999998), (725.1714285714281, 398.7428571428569), (744.8571428571423, 386.7714285714284), (768.1428571428568, 367.34285714285693)], [(849.9999999999995, 304.25714285714264), (833.9999999999995, 300.3999999999998), (803.9999999999995, 292.85714285714266), (755.7142857142853, 325.88571428571413), (713.7714285714281, 356.8857142857141), (709.9999999999997, 277.59999999999985), (631.9999999999997, 246.34285714285699), (584.9999999999997, 225.79999999999987), (530.7142857142853, 222.14285714285703), (709.9999999999997, 316.142857142857), (615.9999999999997, 316.142857142857), (568.9999999999997, 319.99999999999983), (506.9999999999997, 321.0857142857141), (713.8571428571424, 351.74285714285696), (678.4857142857138, 371.94285714285695), (713.9428571428567, 363.05714285714265), (729.8571428571424, 352.0857142857141), (740.9999999999995, 382.9999999999998), (725.1714285714281, 399.08571428571406), (739.7142857142852, 394.3999999999998), (747.5999999999996, 380.17142857142835)], [(849.9999999999995, 304.74285714285696), (833.9999999999995, 306.02857142857124), (803.9999999999995, 300.142857142857), (756.9999999999997, 332.22857142857123), (708.7142857142853, 363.22857142857123), (709.9999999999997, 285.2285714285713), (631.9999999999997, 253.48571428571415), (584.9999999999997, 230.85714285714272), (532.8571428571424, 227.28571428571416), (709.9999999999997, 321.2857142857141), (615.9999999999997, 321.2857142857141), (568.9999999999997, 319.99999999999983), (506.9999999999997, 321.0857142857141), (708.7142857142853, 351.5999999999998), (678.3428571428567, 379.14285714285694), (708.4571428571425, 368.5428571428569), (724.7142857142853, 364.4285714285712), (740.9999999999995, 382.9999999999998), (725.7428571428568, 398.9999999999998), (740.9999999999995, 400.0285714285712), (738.3428571428567, 384.37142857142834)], [(849.9999999999995, 305.0857142857141), (833.9999999999995, 305.0857142857141), (803.9999999999995, 305.2857142857141), (756.9999999999997, 337.2857142857141), (709.9999999999997, 368.2857142857141), (709.9999999999997, 290.2857142857141), (631.9999999999997, 258.1999999999998), (584.9999999999997, 238.142857142857), (539.285714285714, 225.9999999999999), (709.9999999999997, 319.99999999999983), (615.9999999999997, 319.99999999999983), (568.9999999999997, 319.99999999999983), (506.9999999999997, 320.742857142857), (708.7142857142853, 351.3999999999998), (677.828571428571, 384.37142857142834), (709.2571428571424, 367.7428571428569), (725.9999999999995, 368.2857142857141), (740.9999999999995, 382.9999999999998), (726.0857142857138, 398.9999999999998), (740.9999999999995, 399.17142857142835), (736.9714285714281, 384.37142857142834)], [(849.9999999999995, 304.99999999999983), (833.9142857142853, 305.0857142857141), (803.9999999999995, 303.99999999999983), (756.9999999999997, 335.9999999999998), (708.7142857142853, 366.9999999999998), (709.9999999999997, 290.2857142857141), (631.9999999999997, 256.99999999999983), (584.9999999999997, 243.28571428571416), (537.9999999999997, 225.9999999999999), (709.9999999999997, 319.99999999999983), (615.9999999999997, 319.99999999999983), (568.9999999999997, 319.99999999999983), (505.71428571428544, 320.2571428571427), (713.8571428571424, 351.34285714285693), (678.2571428571425, 382.74285714285696), (708.9142857142853, 368.08571428571406), (725.9999999999995, 366.9999999999998), (740.9999999999995, 382.9999999999998), (725.9999999999995, 398.9999999999998), (740.9999999999995, 398.7428571428569), (755.828571428571, 377.5142857142855)], [(849.9999999999995, 304.99999999999983), (834.3428571428567, 304.74285714285696), (803.9999999999995, 303.99999999999983), (756.9999999999997, 335.9999999999998), (713.8571428571424, 366.9999999999998), (709.9999999999997, 285.142857142857), (631.9999999999997, 256.99999999999983), (584.9999999999997, 241.99999999999986), (537.9999999999997, 225.9999999999999), (709.9999999999997, 319.99999999999983), (615.9999999999997, 319.99999999999983), (568.9999999999997, 319.99999999999983), (510.85714285714255, 319.91428571428554), (721.1428571428567, 350.8285714285712), (678.7428571428568, 382.17142857142835), (708.9142857142853, 367.9999999999998), (724.7142857142853, 368.2857142857141), (740.9999999999995, 382.9999999999998), (724.7142857142853, 398.9999999999998), (740.9999999999995, 398.25714285714264), (764.9428571428566, 376.5999999999998)], [(849.9999999999995, 304.99999999999983), (834.3999999999995, 304.25714285714264), (803.9999999999995, 303.99999999999983), (756.9999999999997, 335.9999999999998), (721.1428571428567, 366.9999999999998), (708.7142857142853, 276.57142857142844), (631.9999999999997, 256.99999999999983), (584.9999999999997, 241.99999999999986), (537.9999999999997, 225.9999999999999), (708.7142857142853, 319.99999999999983), (615.9142857142854, 319.99999999999983), (568.9999999999997, 319.99999999999983), (518.1428571428569, 319.99999999999983), (726.2857142857139, 351.25714285714264), (679.085714285714, 382.17142857142835), (709.2571428571424, 367.9999999999998), (729.8571428571424, 363.14285714285694), (740.9999999999995, 382.9999999999998), (729.8571428571424, 398.9999999999998), (740.9999999999995, 397.9142857142855), (776.4857142857138, 373.3999999999998)], [(849.9999999999995, 304.99999999999983), (834.5999999999996, 303.91428571428554), (802.7142857142852, 305.2857142857141), (756.9999999999997, 335.9999999999998), (726.2857142857139, 366.9999999999998), (713.8571428571424, 276.57142857142844), (631.9999999999997, 256.99999999999983), (584.9999999999997, 243.28571428571416), (537.9999999999997, 225.9999999999999), (713.8571428571424, 319.99999999999983), (616.2571428571425, 319.99999999999983), (568.9999999999997, 319.99999999999983), (523.285714285714, 319.99999999999983), (724.9142857142854, 351.74285714285696), (678.9999999999995, 382.74285714285696), (709.7428571428568, 368.08571428571406), (737.1428571428567, 355.85714285714266), (740.9999999999995, 382.9999999999998), (737.1428571428567, 398.9999999999998), (739.7142857142852, 399.2857142857141), (777.028571428571, 372.48571428571404)], [(849.9999999999995, 304.99999999999983), (834.6571428571424, 305.1999999999998), (807.8571428571423, 300.142857142857), (755.7142857142853, 337.2857142857141), (724.9999999999995, 366.9999999999998), (721.1428571428567, 286.4285714285712), (630.7142857142853, 256.99999999999983), (584.9999999999997, 238.142857142857), (537.9999999999997, 225.9999999999999), (721.1428571428567, 319.99999999999983), (615.5428571428567, 319.99999999999983), (568.9999999999997, 319.99999999999983), (521.9999999999997, 319.99999999999983), (725.2571428571425, 352.1714285714284), (678.9999999999995, 384.37142857142834), (710.085714285714, 367.7428571428569), (742.2857142857138, 350.71428571428555), (740.9999999999995, 382.9999999999998), (742.2857142857138, 398.9999999999998), (746.1428571428567, 394.14285714285694), (790.7428571428567, 365.62857142857126)], [(849.9142857142851, 305.0857142857141), (835.0857142857138, 300.3999999999998), (815.1428571428567, 292.85714285714266), (760.8571428571424, 332.14285714285694), (724.9999999999995, 366.9999999999998), (726.2857142857139, 286.4285714285712), (635.8571428571424, 258.1999999999998), (583.7142857142853, 230.85714285714272), (537.9999999999997, 227.28571428571416), (726.2857142857139, 319.99999999999983), (620.5999999999997, 321.2857142857141), (568.9142857142854, 321.2857142857141), (521.9999999999997, 319.99999999999983), (725.7428571428568, 351.74285714285696), (678.9999999999995, 379.14285714285694), (709.9999999999997, 367.25714285714264), (740.9999999999995, 351.99999999999983), (740.9999999999995, 382.9999999999998), (740.9999999999995, 399.08571428571406), (748.2857142857138, 386.85714285714266), (787.9999999999995, 366.9999999999998)], [(850.2571428571423, 304.74285714285696), (834.9999999999995, 293.6857142857141), (820.2857142857138, 287.71428571428555), (768.1428571428568, 324.85714285714266), (724.9999999999995, 368.2857142857141), (724.9999999999995, 277.85714285714266), (643.1428571428568, 253.48571428571415), (588.8571428571424, 225.7142857142856), (537.9999999999997, 222.14285714285703), (724.9999999999995, 321.2857142857141), (627.6571428571425, 316.142857142857), (569.2571428571425, 316.142857142857), (521.9999999999997, 319.99999999999983), (724.7999999999995, 352.542857142857), (678.9999999999995, 371.85714285714266), (709.9999999999997, 366.9142857142855), (740.9999999999995, 351.99999999999983), (740.9999999999995, 382.9999999999998), (740.9999999999995, 398.7428571428569), (744.8571428571423, 381.7142857142855), (787.9999999999995, 366.9999999999998)], [(849.5428571428566, 305.4571428571427), (834.9999999999995, 288.542857142857), (818.9999999999995, 288.9999999999999), (773.2857142857139, 319.71428571428555), (724.9999999999995, 363.14285714285694), (724.9999999999995, 272.79999999999984), (648.2857142857139, 246.34285714285699), (596.1428571428568, 227.08571428571418), (537.9999999999997, 214.85714285714275), (724.9999999999995, 316.142857142857), (632.9428571428568, 308.85714285714266), (568.5428571428567, 308.85714285714266), (521.9142857142854, 319.99999999999983), (729.8571428571424, 345.7714285714284), (677.7142857142853, 366.7999999999998), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.99999999999983), (739.7142857142852, 384.28571428571405), (740.9999999999995, 398.25714285714264), (744.8571428571423, 382.9999999999998), (787.9999999999995, 366.9999999999998)], [(854.6857142857137, 300.3142857142855), (834.9999999999995, 289.2571428571427), (818.9999999999995, 288.9999999999999), (771.9999999999995, 321.0857142857141), (724.9999999999995, 355.85714285714266), (724.9999999999995, 273.742857142857), (646.9999999999995, 241.0571428571427), (601.2857142857139, 226.74285714285702), (537.9999999999997, 209.71428571428558), (724.9999999999995, 308.85714285714266), (632.085714285714, 303.71428571428555), (573.685714285714, 303.71428571428555), (522.2571428571425, 319.99999999999983), (737.1428571428567, 343.6285714285712), (682.8571428571424, 367.7428571428569), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.99999999999983), (746.1428571428567, 377.85714285714266), (740.9999999999995, 397.9142857142855), (746.9999999999995, 382.9999999999998), (786.7142857142852, 368.2857142857141)], [(861.3999999999994, 293.59999999999985), (834.9999999999995, 288.9142857142856), (818.9999999999995, 288.9999999999999), (771.9999999999995, 320.742857142857), (724.9142857142854, 350.71428571428555), (724.9999999999995, 273.2571428571427), (646.9999999999995, 241.91428571428557), (599.9999999999995, 226.25714285714272), (537.9999999999997, 210.99999999999986), (724.9999999999995, 303.71428571428555), (631.9999999999997, 304.99999999999983), (580.3999999999996, 304.99999999999983), (522.7428571428568, 319.99999999999983), (742.2857142857138, 346.1999999999998), (690.1428571428568, 367.25714285714264), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.99999999999983), (748.2857142857138, 375.71428571428544), (740.9999999999995, 397.9999999999998), (749.9142857142854, 382.9999999999998), (791.8571428571424, 363.14285714285694)], [(867.3999999999995, 287.59999999999985), (834.9999999999995, 288.9999999999999), (818.9999999999995, 288.9999999999999), (771.9999999999995, 320.2571428571427), (725.3428571428567, 351.99999999999983), (724.9999999999995, 272.91428571428554), (646.9999999999995, 241.99999999999986), (599.9999999999995, 225.9142857142856), (537.9999999999997, 210.99999999999986), (724.9999999999995, 304.99999999999983), (631.9999999999997, 304.99999999999983), (585.1999999999997, 304.99999999999983), (523.085714285714, 319.99999999999983), (740.9999999999995, 352.7714285714284), (695.2857142857139, 366.9142857142855), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.99999999999983), (746.1428571428567, 377.85714285714266), (740.9999999999995, 397.9999999999998), (751.1999999999995, 382.9999999999998), (799.1428571428567, 355.85714285714266)], [(861.3999999999994, 294.79999999999984), (835.0857142857138, 288.9999999999999), (818.9999999999995, 288.9999999999999), (771.9999999999995, 319.91428571428554), (725.4857142857138, 351.99999999999983), (724.9999999999995, 272.99999999999983), (646.9999999999995, 241.99999999999986), (599.9999999999995, 225.9999999999999), (537.9999999999997, 210.99999999999986), (724.9999999999995, 304.99999999999983), (631.9999999999997, 304.99999999999983), (585.1999999999997, 304.99999999999983), (522.9999999999997, 319.99999999999983), (740.9999999999995, 351.34285714285693), (693.9999999999995, 366.9999999999998), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.99999999999983), (739.7142857142852, 384.28571428571405), (740.9999999999995, 397.9999999999998), (757.7714285714281, 382.9999999999998), (804.2857142857138, 350.71428571428555)], [(854.6857142857137, 296.79999999999984), (834.7428571428567, 288.9999999999999), (818.9999999999995, 288.9999999999999), (771.9999999999995, 319.99999999999983), (725.3428571428567, 351.99999999999983), (724.9999999999995, 272.99999999999983), (646.9999999999995, 241.99999999999986), (599.9999999999995, 225.9999999999999), (537.9999999999997, 210.99999999999986), (724.9999999999995, 304.99999999999983), (631.9999999999997, 304.99999999999983), (580.3999999999996, 304.99999999999983), (523.085714285714, 319.99999999999983), (742.2857142857138, 350.9142857142855), (693.9999999999995, 366.9999999999998), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.99999999999983), (740.9999999999995, 382.9999999999998), (740.9999999999995, 397.9999999999998), (756.3428571428567, 382.9999999999998), (804.2857142857138, 351.99999999999983)], [(849.5428571428566, 294.88571428571413), (834.2571428571424, 288.9999999999999), (820.2857142857138, 288.9999999999999), (773.2857142857139, 319.99999999999983), (724.9142857142854, 351.99999999999983), (724.9999999999995, 272.99999999999983), (646.9999999999995, 241.99999999999986), (601.2857142857139, 225.9999999999999), (537.9999999999997, 210.99999999999986), (724.9999999999995, 304.99999999999983), (631.9999999999997, 304.99999999999983), (573.5999999999997, 304.99999999999983), (522.7428571428568, 319.99999999999983), (737.2285714285711, 352.2857142857141), (695.2857142857139, 366.9999999999998), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.99999999999983), (740.9999999999995, 382.9999999999998), (740.9999999999995, 397.9999999999998), (757.1999999999996, 382.9999999999998), (799.1428571428567, 351.99999999999983)], [(850.2571428571423, 287.2571428571427), (833.9142857142853, 288.9999999999999), (815.1428571428567, 288.9999999999999), (768.1428571428568, 319.99999999999983), (724.9999999999995, 351.99999999999983), (724.9999999999995, 272.99999999999983), (646.9999999999995, 241.99999999999986), (596.1428571428568, 225.9999999999999), (537.9999999999997, 210.99999999999986), (724.9999999999995, 304.99999999999983), (631.9999999999997, 304.99999999999983), (568.7999999999996, 304.99999999999983), (522.2571428571425, 319.99999999999983), (729.5142857142853, 345.85714285714266), (690.1428571428568, 366.9142857142855), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.99999999999983), (740.9999999999995, 382.9999999999998), (742.2857142857138, 397.9999999999998), (752.1428571428567, 382.9999999999998), (791.8571428571424, 351.99999999999983)], [(849.9142857142851, 293.11428571428553), (835.2857142857139, 288.9999999999999), (807.8571428571423, 288.9999999999999), (759.5714285714282, 319.99999999999983), (724.9999999999995, 351.99999999999983), (726.2857142857139, 272.99999999999983), (646.9999999999995, 241.99999999999986), (588.8571428571424, 225.9999999999999), (537.9999999999997, 210.99999999999986), (726.2857142857139, 304.99999999999983), (631.9999999999997, 304.99999999999983), (569.9999999999997, 304.99999999999983), (521.9142857142854, 319.99999999999983), (724.3142857142852, 343.6285714285712), (682.8571428571424, 367.25714285714264), (709.9999999999997, 366.9999999999998), (740.9999999999995, 352.0857142857141), (740.9999999999995, 382.9999999999998), (735.9428571428567, 397.9999999999998), (744.8571428571423, 382.9999999999998), (786.7142857142852, 351.99999999999983)], [(849.9999999999995, 300.0571428571427), (828.8571428571424, 288.9999999999999), (802.7142857142852, 288.9999999999999), (759.5714285714282, 319.99999999999983), (724.9999999999995, 352.0857142857141), (719.8571428571424, 272.99999999999983), (646.9999999999995, 241.99999999999986), (582.4285714285711, 225.9999999999999), (537.9999999999997, 210.99999999999986), (719.8571428571424, 304.99999999999983), (631.9999999999997, 304.99999999999983), (568.7999999999996, 304.99999999999983), (521.9142857142854, 319.99999999999983), (725.3142857142852, 346.1999999999998), (676.428571428571, 367.8285714285712), (709.9999999999997, 366.9999999999998), (740.9999999999995, 351.74285714285696), (740.9999999999995, 382.9999999999998), (733.3714285714282, 397.9999999999998), (739.7142857142852, 382.9999999999998), (787.9999999999995, 351.99999999999983)], [(849.9142857142851, 305.1999999999998), (826.7142857142853, 288.9999999999999), (802.7142857142852, 288.9999999999999), (768.1428571428568, 319.99999999999983), (724.9999999999995, 351.74285714285696), (717.7142857142853, 272.99999999999983), (646.9999999999995, 241.99999999999986), (588.8571428571424, 225.9999999999999), (537.9999999999997, 210.99999999999986), (717.7142857142853, 304.99999999999983), (631.9999999999997, 304.99999999999983), (573.5142857142854, 304.99999999999983), (522.2571428571425, 319.99999999999983), (725.5999999999996, 352.7714285714284), (682.8571428571424, 367.8285714285712), (708.7142857142853, 368.2857142857141), (740.9999999999995, 352.4571428571427), (740.9999999999995, 382.9999999999998), (735.3714285714282, 397.9999999999998), (739.7142857142852, 382.9999999999998), (787.9999999999995, 351.99999999999983)], [(850.3428571428567, 304.25714285714264), (828.8571428571424, 288.9999999999999), (807.8571428571423, 290.2857142857141), (773.2857142857139, 321.2857142857141), (724.9999999999995, 353.8285714285712), (719.8571428571424, 272.99999999999983), (646.9999999999995, 241.99999999999986), (596.1428571428568, 225.9999999999999), (537.9999999999997, 210.99999999999986), (719.8571428571424, 304.99999999999983), (631.9999999999997, 304.99999999999983), (580.6571428571425, 304.99999999999983), (522.7428571428568, 319.99999999999983), (724.3714285714282, 351.25714285714264), (690.1428571428568, 367.25714285714264), (712.4857142857138, 363.14285714285694), (738.3428571428567, 347.3999999999998), (740.9999999999995, 382.9999999999998), (741.9428571428567, 397.9142857142855), (744.771428571428, 382.9999999999998), (787.9999999999995, 351.99999999999983)], [(850.4857142857138, 304.74285714285696), (835.1999999999996, 288.9999999999999), (815.1428571428567, 285.2285714285713), (771.9999999999995, 317.5142857142855), (724.9142857142854, 343.2857142857141), (726.2857142857139, 272.99999999999983), (646.9999999999995, 241.99999999999986), (601.2857142857139, 225.9142857142856), (536.7142857142853, 210.99999999999986), (726.2857142857139, 304.99999999999983), (631.9999999999997, 304.99999999999983), (585.9428571428568, 303.71428571428555), (523.085714285714, 319.99999999999983), (731.2285714285711, 351.17142857142835), (695.2857142857139, 366.9142857142855), (725.2571428571425, 355.85714285714266), (748.8857142857139, 340.34285714285693), (739.7142857142852, 382.9999999999998), (741.0857142857138, 398.25714285714264), (751.1142857142852, 382.9999999999998), (786.7142857142852, 350.71428571428555)], [(850.2571428571423, 305.0857142857141), (834.3428571428567, 288.9999999999999), (820.2857142857138, 277.59999999999985), (773.2857142857139, 306.02857142857124), (725.2571428571425, 329.74285714285696), (724.9999999999995, 272.99999999999983), (646.9999999999995, 241.99999999999986), (599.9999999999995, 226.25714285714272), (541.8571428571425, 209.71428571428558), (724.9999999999995, 303.71428571428555), (631.9999999999997, 303.71428571428555), (585.085714285714, 308.85714285714266), (522.9999999999997, 319.91428571428554), (731.9999999999995, 351.74285714285696), (695.2857142857139, 366.9142857142855), (738.1714285714281, 350.71428571428555), (764.3714285714282, 335.0571428571427), (744.771428571428, 381.7142857142855), (740.9999999999995, 398.7428571428569), (761.8857142857139, 382.9999999999998), (793.1428571428567, 355.85714285714266)], [(850.1714285714281, 304.99999999999983), (834.4857142857138, 288.9999999999999), (818.9999999999995, 273.2571428571427), (768.1428571428568, 289.342857142857), (725.7428571428568, 313.8285714285712), (724.9999999999995, 272.91428571428554), (646.9999999999995, 240.71428571428558), (599.9999999999995, 225.4571428571427), (549.1428571428569, 214.85714285714275), (724.9999999999995, 308.85714285714266), (631.9999999999997, 308.85714285714266), (584.9999999999997, 316.142857142857), (522.9999999999997, 320.2571428571427), (734.9999999999995, 350.79999999999984), (690.2285714285711, 365.97142857142836), (746.3999999999995, 350.71428571428555), (775.1428571428567, 335.9142857142855), (751.1142857142852, 386.7714285714284), (742.2857142857138, 397.7999999999997), (770.8857142857139, 380.34285714285693), (797.9428571428566, 360.4857142857141)], [(850.828571428571, 303.71428571428555), (834.3428571428567, 290.2857142857141), (820.2857142857138, 269.142857142857), (760.8571428571424, 273.742857142857), (724.7999999999995, 314.1714285714284), (724.9999999999995, 273.2571428571427), (646.9999999999995, 245.77142857142843), (599.9999999999995, 230.942857142857), (555.5714285714282, 222.14285714285703), (724.9999999999995, 316.142857142857), (631.9999999999997, 316.142857142857), (585.085714285714, 321.2857142857141), (522.9999999999997, 320.742857142857), (737.2285714285711, 355.85714285714266), (683.8857142857139, 371.5999999999998), (731.5714285714281, 354.4857142857141), (781.6857142857139, 331.97142857142836), (764.5428571428567, 394.3999999999998), (739.8857142857139, 401.48571428571404), (770.7142857142853, 388.2285714285712), (793.228571428571, 370.85714285714266)], [(850.7428571428566, 308.85714285714266), (833.9142857142853, 285.142857142857), (815.1428571428567, 262.88571428571413), (755.7999999999996, 258.0857142857141), (733.8857142857139, 291.542857142857), (724.9999999999995, 273.742857142857), (646.9999999999995, 253.39999999999986), (599.9999999999995, 238.142857142857), (549.1428571428569, 227.19999999999987), (724.9999999999995, 321.1999999999998), (631.9999999999997, 321.2857142857141), (585.9428571428568, 319.91428571428554), (523.085714285714, 319.79999999999984), (738.1714285714281, 363.14285714285694), (674.3999999999996, 379.22857142857123), (702.1428571428567, 365.97142857142836), (751.9428571428567, 345.4285714285712), (760.2571428571424, 400.0285714285712), (721.6285714285709, 414.25714285714264), (734.0571428571425, 414.25714285714264), (742.1714285714281, 406.1428571428569)], [(849.3142857142852, 316.0571428571427), (833.9999999999995, 276.57142857142844), (807.9428571428566, 252.1142857142856), (758.028571428571, 240.6285714285713), (725.0571428571425, 247.39999999999986), (724.9999999999995, 272.79999999999984), (646.9999999999995, 259.0285714285713), (601.2857142857139, 243.28571428571416), (541.8571428571425, 226.25714285714272), (724.9999999999995, 320.2571428571427), (631.9999999999997, 319.99999999999983), (580.7428571428568, 320.2571428571427), (522.7428571428568, 324.85714285714266), (729.1142857142853, 368.2857142857141), (662.9142857142854, 384.28571428571405), (656.2285714285711, 384.02857142857124), (668.2285714285711, 380.17142857142835), (750.2571428571425, 399.08571428571406), (700.9428571428567, 427.1714285714283), (688.9714285714282, 439.1428571428569), (677.9142857142854, 447.71428571428544)], [(854.2571428571423, 320.2571428571427), (833.9999999999995, 276.57142857142844), (802.4571428571423, 245.77142857142843), (752.3999999999995, 230.0285714285713), (707.3714285714282, 197.91428571428557), (724.9999999999995, 277.85714285714266), (648.2857142857139, 258.0857142857141), (596.1428571428568, 241.99999999999986), (536.7142857142853, 226.74285714285702), (726.2857142857139, 319.4571428571427), (632.085714285714, 319.99999999999983), (573.2571428571425, 320.742857142857), (523.5428571428569, 332.14285714285694), (723.628571428571, 366.9999999999998), (650.6571428571425, 382.9999999999998), (622.5999999999997, 396.7999999999998), (580.9999999999997, 406.2857142857141), (738.3428571428567, 398.9999999999998), (692.5428571428567, 431.37142857142834), (657.6857142857139, 453.885714285714), (629.085714285714, 467.2285714285712)], [(861.4857142857138, 324.5999999999998), (833.9999999999995, 285.142857142857), (804.5428571428567, 240.71428571428558), (744.771428571428, 220.77142857142846), (689.9714285714281, 180.85714285714278), (726.2857142857139, 285.142857142857), (643.1428571428568, 257.9999999999999), (588.8571428571424, 241.99999999999986), (537.9999999999997, 227.08571428571418), (721.1428571428567, 324.94285714285695), (632.9428571428568, 319.99999999999983), (568.0571428571425, 321.1714285714284), (518.0571428571426, 337.2857142857141), (724.9999999999995, 366.9999999999998), (646.4571428571426, 382.9999999999998), (596.2285714285711, 395.4285714285712), (545.1999999999996, 406.5428571428569), (741.0857142857138, 397.71428571428544), (690.2285714285711, 429.9999999999998), (651.5999999999997, 453.285714285714), (613.5999999999997, 461.82857142857114)], [(866.1999999999996, 332.22857142857123), (835.2857142857139, 290.2857142857141), (799.0571428571425, 241.99999999999986), (739.7142857142852, 214.85714285714275), (693.9999999999995, 174.11428571428561), (721.1428571428567, 290.2857142857141), (635.8571428571424, 257.9999999999999), (583.7142857142853, 241.99999999999986), (537.9999999999997, 226.9999999999999), (713.8571428571424, 332.14285714285694), (627.7428571428568, 319.99999999999983), (568.9142857142854, 320.742857142857), (510.85714285714255, 337.2857142857141), (726.2857142857139, 366.9999999999998), (647.2571428571425, 382.9999999999998), (587.4857142857139, 386.85714285714266), (540.485714285714, 393.11428571428553), (741.9428571428567, 402.85714285714266), (682.5999999999997, 429.9999999999998), (642.1142857142853, 451.1428571428569), (603.5714285714282, 461.2571428571425)], [(864.9142857142853, 337.2857142857141), (830.1428571428568, 288.9999999999999), (791.8571428571424, 241.99999999999986), (740.9999999999995, 209.71428571428558), (693.9999999999995, 167.7714285714285), (713.8571428571424, 288.9999999999999), (630.7142857142853, 257.9999999999999), (584.9999999999997, 241.99999999999986), (537.9999999999997, 226.9999999999999), (708.7142857142853, 337.2857142857141), (620.2571428571425, 319.99999999999983), (568.9999999999997, 320.2571428571427), (505.71428571428544, 332.14285714285694), (721.1428571428567, 366.9999999999998), (648.1999999999996, 382.9999999999998), (583.7142857142853, 381.7142857142855), (536.7142857142853, 386.7714285714284), (736.6571428571424, 410.14285714285694), (676.9714285714282, 429.9999999999998), (635.7714285714281, 444.71428571428544), (599.4571428571426, 460.9142857142855)], [(865.2571428571424, 335.9999999999998), (821.5714285714282, 288.9999999999999), (786.7142857142852, 241.99999999999986), (740.9999999999995, 210.99999999999986), (693.9999999999995, 162.7142857142856), (708.7142857142853, 288.9999999999999), (631.9999999999997, 257.9999999999999), (584.9999999999997, 241.99999999999986), (537.9999999999997, 226.9999999999999), (709.9999999999997, 335.9999999999998), (615.0571428571425, 319.99999999999983), (568.9999999999997, 319.91428571428554), (506.9999999999997, 324.85714285714266), (712.5714285714282, 366.9999999999998), (641.8571428571424, 382.9999999999998), (584.9999999999997, 382.9999999999998), (537.9999999999997, 381.7142857142855), (729.5142857142853, 415.28571428571405), (677.9142857142854, 429.9999999999998), (630.7142857142853, 445.9999999999997), (600.2571428571425, 460.9999999999998)], [(865.7428571428567, 335.9999999999998), (821.5714285714282, 288.9999999999999), (787.9999999999995, 241.99999999999986), (740.9999999999995, 211.08571428571415), (693.9999999999995, 163.9999999999999), (708.7142857142853, 288.9999999999999), (631.9999999999997, 257.9999999999999), (584.9999999999997, 241.99999999999986), (537.9999999999997, 226.9999999999999), (709.9999999999997, 335.9999999999998), (614.6285714285711, 319.91428571428554), (568.9999999999997, 319.99999999999983), (506.9999999999997, 319.71428571428555), (712.5714285714282, 366.9999999999998), (639.7142857142853, 382.9999999999998), (584.9999999999997, 382.9999999999998), (537.9999999999997, 382.9999999999998), (723.5142857142853, 413.9999999999998), (677.9999999999995, 429.9999999999998), (631.9999999999997, 445.9999999999997), (599.828571428571, 460.9999999999998)], [(866.0857142857137, 335.9999999999998), (830.1428571428568, 288.9999999999999), (786.7142857142852, 241.99999999999986), (740.9999999999995, 210.74285714285702), (693.9999999999995, 163.9999999999999), (713.8571428571424, 288.9999999999999), (630.7142857142853, 257.9999999999999), (584.9999999999997, 241.99999999999986), (537.9999999999997, 226.9999999999999), (708.7142857142853, 335.9999999999998), (619.7714285714283, 320.2571428571427), (568.9999999999997, 319.99999999999983), (505.71428571428544, 320.99999999999983), (721.1428571428567, 366.9999999999998), (641.8571428571424, 382.9999999999998), (584.9999999999997, 382.9999999999998), (537.9999999999997, 382.9999999999998), (729.8571428571424, 413.9999999999998), (677.9142857142854, 429.9999999999998), (630.7142857142853, 445.9999999999997), (600.2571428571425, 460.9999999999998)], [(865.9999999999994, 335.9999999999998), (835.1999999999996, 288.9999999999999), (791.8571428571424, 241.99999999999986), (740.9999999999995, 210.25714285714275), (693.9999999999995, 163.9999999999999), (721.1428571428567, 288.9999999999999), (635.8571428571424, 257.9999999999999), (583.7142857142853, 241.99999999999986), (537.9999999999997, 226.9999999999999), (713.8571428571424, 335.9999999999998), (627.3999999999996, 320.742857142857), (568.9999999999997, 319.99999999999983), (510.85714285714255, 319.71428571428555), (726.2857142857139, 366.9999999999998), (648.2857142857139, 382.9999999999998), (584.9999999999997, 382.9999999999998), (537.9999999999997, 382.9999999999998), (737.1428571428567, 413.9999999999998), (678.2571428571425, 429.9999999999998), (635.8571428571424, 445.9999999999997), (600.7428571428568, 460.9999999999998)], [(865.9999999999994, 335.9999999999998), (834.2571428571424, 288.9999999999999), (799.1428571428567, 241.99999999999986), (740.9999999999995, 209.9142857142856), (693.9142857142854, 163.9999999999999), (726.2857142857139, 288.9999999999999), (643.1428571428568, 257.9999999999999), (588.8571428571424, 241.99999999999986), (537.9999999999997, 226.9999999999999), (721.1428571428567, 335.9999999999998), (633.028571428571, 321.0857142857141), (568.9142857142854, 319.99999999999983), (518.1428571428569, 324.85714285714266), (724.9142857142854, 366.9142857142855), (646.9142857142854, 382.9999999999998), (583.7142857142853, 382.9999999999998), (537.9999999999997, 382.9999999999998), (742.2857142857138, 413.9999999999998), (678.7428571428568, 429.9999999999998), (643.1428571428568, 445.9999999999997), (601.085714285714, 460.9999999999998)], [(865.9999999999994, 335.9999999999998), (834.7428571428567, 288.9999999999999), (804.2857142857138, 241.99999999999986), (740.9999999999995, 209.9999999999999), (693.0571428571425, 163.9999999999999), (724.9999999999995, 288.9999999999999), (648.2857142857139, 257.9999999999999), (596.1428571428568, 241.99999999999986), (536.7142857142853, 226.9999999999999), (726.2857142857139, 335.9999999999998), (632.085714285714, 320.99999999999983), (569.2571428571425, 319.99999999999983), (523.285714285714, 332.14285714285694), (725.2571428571425, 367.25714285714264), (647.2571428571425, 382.9999999999998), (588.8571428571424, 382.9999999999998), (536.7142857142853, 382.9999999999998), (740.9999999999995, 413.9999999999998), (679.085714285714, 429.9999999999998), (648.2857142857139, 445.9999999999997), (599.7142857142853, 460.9999999999998)], [(865.9999999999994, 335.9999999999998), (835.0857142857138, 288.9999999999999), (802.9999999999995, 241.99999999999986), (739.7142857142852, 209.9142857142856), (698.3428571428568, 163.9999999999999), (724.9142857142854, 288.9999999999999), (646.9999999999995, 257.9999999999999), (601.2857142857139, 241.99999999999986), (541.8571428571425, 226.9999999999999), (724.9999999999995, 335.9999999999998), (631.9999999999997, 320.99999999999983), (568.5428571428567, 319.91428571428554), (521.9999999999997, 337.2857142857141), (725.7428571428568, 367.7428571428569), (647.7428571428568, 382.9999999999998), (596.1428571428568, 382.9999999999998), (541.8571428571425, 382.9999999999998), (740.9999999999995, 413.9999999999998), (677.7142857142853, 429.9999999999998), (646.9999999999995, 445.9999999999997), (604.8571428571425, 460.9999999999998)], [(865.9999999999994, 335.9999999999998), (834.9999999999995, 288.9999999999999), (802.9142857142853, 241.99999999999986), (744.8571428571423, 210.25714285714275), (705.4857142857138, 163.9999999999999), (725.3428571428567, 288.9999999999999), (646.9999999999995, 257.9999999999999), (599.9999999999995, 241.99999999999986), (549.1428571428569, 226.9999999999999), (724.9999999999995, 335.9999999999998), (631.9999999999997, 320.99999999999983), (573.685714285714, 320.342857142857), (521.9142857142854, 335.9999999999998), (726.0857142857138, 368.17142857142835), (648.085714285714, 382.9999999999998), (601.2857142857139, 382.9999999999998), (549.1428571428569, 382.9999999999998), (740.9999999999995, 413.9999999999998), (682.8571428571424, 429.9999999999998), (646.9999999999995, 445.9999999999997), (612.1428571428569, 460.9999999999998)], [(865.9999999999994, 335.9999999999998), (834.9999999999995, 288.9999999999999), (803.2571428571424, 241.99999999999986), (752.1428571428567, 210.8285714285713), (710.1999999999996, 163.9999999999999), (725.3999999999996, 288.9999999999999), (646.9999999999995, 257.9999999999999), (599.9999999999995, 241.99999999999986), (554.285714285714, 226.9999999999999), (724.9999999999995, 335.9999999999998), (631.9999999999997, 321.0857142857141), (580.3142857142853, 320.4857142857141), (522.2571428571425, 337.2857142857141), (725.9999999999995, 367.7428571428569), (647.9999999999997, 382.9999999999998), (599.9999999999995, 382.9999999999998), (554.1999999999996, 382.9999999999998), (740.9999999999995, 413.9999999999998), (690.1428571428568, 429.9999999999998), (646.9999999999995, 445.9999999999997), (617.285714285714, 460.9999999999998)], [(865.9999999999994, 337.2857142857141), (834.9999999999995, 288.9999999999999), (803.7428571428567, 241.99999999999986), (757.2857142857139, 210.8285714285713), (708.9999999999995, 163.9999999999999), (725.5999999999996, 288.9999999999999), (646.9999999999995, 257.9999999999999), (599.9999999999995, 241.99999999999986), (552.9999999999997, 226.9999999999999), (724.9999999999995, 335.9999999999998), (631.9999999999997, 320.742857142857), (585.4571428571426, 320.342857142857), (522.7428571428568, 332.14285714285694), (725.9999999999995, 367.25714285714264), (647.9999999999997, 382.9999999999998), (599.9999999999995, 382.9999999999998), (553.2571428571425, 382.9999999999998), (740.9999999999995, 415.28571428571405), (695.2857142857139, 429.9999999999998), (646.9999999999995, 445.9999999999997), (615.9999999999997, 460.9999999999998)], [(865.9999999999994, 332.22857142857123), (834.9999999999995, 290.2857142857141), (804.0857142857138, 241.99999999999986), (755.9999999999995, 210.25714285714275), (708.9999999999995, 163.9999999999999), (725.6571428571424, 288.9999999999999), (646.9142857142854, 257.9999999999999), (599.9999999999995, 241.99999999999986), (552.9999999999997, 227.08571428571418), (724.9999999999995, 337.2857142857141), (631.9999999999997, 320.2571428571427), (584.7428571428568, 319.91428571428554), (523.085714285714, 324.85714285714266), (725.9999999999995, 366.9142857142855), (646.7142857142853, 382.9999999999998), (599.9142857142854, 382.9999999999998), (553.7428571428568, 382.9999999999998), (739.7142857142852, 410.14285714285694), (693.9999999999995, 429.9999999999998), (646.9142857142854, 446.085714285714), (615.9999999999997, 460.9999999999998)], [(865.9999999999994, 324.51428571428556), (834.9999999999995, 285.142857142857), (803.9999999999995, 241.99999999999986), (755.9999999999995, 209.9142857142856), (708.9142857142853, 163.9999999999999), (726.0857142857138, 288.9999999999999), (647.2571428571425, 257.9999999999999), (599.9142857142854, 241.99999999999986), (552.9142857142854, 226.74285714285702), (724.9999999999995, 332.14285714285694), (631.9999999999997, 319.91428571428554), (585.085714285714, 319.99999999999983), (522.9999999999997, 319.71428571428555), (724.7142857142853, 366.9999999999998), (651.8571428571424, 382.9999999999998), (600.2571428571425, 382.9999999999998), (554.085714285714, 382.9999999999998), (744.8571428571423, 402.85714285714266), (693.9999999999995, 429.9999999999998), (647.2571428571425, 445.6571428571426), (615.9999999999997, 460.9999999999998)], [(865.9999999999994, 317.94285714285695), (834.9999999999995, 276.57142857142844), (803.9999999999995, 241.99999999999986), (755.9999999999995, 209.9142857142856), (709.2571428571424, 163.9999999999999), (725.9999999999995, 288.9999999999999), (647.7428571428568, 257.9999999999999), (600.2571428571425, 241.99999999999986), (553.2571428571425, 226.17142857142844), (724.9142857142854, 323.5714285714284), (630.7142857142853, 319.99999999999983), (584.9999999999997, 319.99999999999983), (522.9999999999997, 320.99999999999983), (729.8571428571424, 366.9999999999998), (659.1428571428568, 382.9999999999998), (600.7428571428568, 382.9999999999998), (553.9999999999998, 382.9999999999998), (752.1428571428567, 397.71428571428544), (693.9999999999995, 429.9999999999998), (647.7428571428568, 445.51428571428545), (615.9999999999997, 460.9999999999998)], [(865.9999999999994, 324.51428571428556), (834.9999999999995, 276.57142857142844), (803.9999999999995, 241.99999999999986), (755.9999999999995, 210.25714285714275), (709.7428571428568, 163.9999999999999), (725.9999999999995, 288.9999999999999), (648.085714285714, 257.9999999999999), (600.7428571428568, 241.99999999999986), (553.7428571428568, 226.17142857142844), (725.2571428571425, 324.85714285714266), (635.8571428571424, 319.99999999999983), (584.9999999999997, 319.99999999999983), (522.9999999999997, 320.99999999999983), (737.1428571428567, 366.9999999999998), (664.2857142857139, 382.9999999999998), (601.085714285714, 382.9999999999998), (553.9999999999998, 382.9999999999998), (757.2857142857139, 398.9999999999998), (693.9999999999995, 429.9999999999998), (648.085714285714, 445.6571428571426), (615.9999999999997, 460.9999999999998)], [(865.9999999999994, 333.5999999999998), (834.9999999999995, 286.4285714285712), (803.9999999999995, 241.99999999999986), (755.9999999999995, 210.8285714285713), (710.1714285714281, 163.9999999999999), (725.9999999999995, 288.9999999999999), (647.9999999999997, 257.9999999999999), (601.085714285714, 241.99999999999986), (554.085714285714, 226.8285714285713), (725.828571428571, 328.2857142857141), (644.4285714285711, 319.99999999999983), (584.9999999999997, 319.99999999999983), (522.9999999999997, 320.99999999999983), (742.3714285714282, 366.9999999999998), (663.085714285714, 382.9999999999998), (600.9999999999997, 382.9999999999998), (553.9999999999998, 382.9999999999998), (755.9999999999995, 398.9999999999998), (693.9999999999995, 429.9999999999998), (647.9999999999997, 446.085714285714), (615.9999999999997, 460.9999999999998)], [(865.9999999999995, 331.5999999999999), (834.9999999999993, 284.7142857142855), (803.9999999999995, 241.99999999999986), (755.9999999999993, 210.71428571428558), (709.8857142857136, 163.99999999999991), (725.9999999999998, 288.99999999999983), (647.9999999999998, 257.9999999999999), (601.1428571428571, 241.99999999999986), (554.142857142857, 226.71428571428558), (725.7142857142853, 326.14285714285694), (642.7142857142857, 319.99999999999966), (584.9999999999997, 319.99999999999966), (522.9999999999998, 320.99999999999983), (742.885714285714, 366.9999999999997), (662.7428571428572, 382.9999999999998), (600.9999999999999, 382.9999999999998), (553.9999999999999, 382.9999999999998), (755.9999999999993, 398.9999999999998), (693.9999999999995, 429.99999999999966), (647.9999999999998, 446.142857142857), (615.9999999999998, 460.99999999999983)], [(866.0000000000001, 322.6), (834.9999999999994, 276.5714285714284), (803.9999999999999, 241.9999999999999), (755.9999999999995, 210.1714285714285), (709.028571428571, 163.9999999999999), (725.9999999999998, 288.99999999999983), (647.9999999999999, 258.00000000000006), (600.9142857142858, 241.9999999999999), (553.9142857142858, 226.17142857142858), (725.1714285714286, 319.7142857142856), (634.5714285714289, 319.9999999999998), (584.9999999999998, 319.9999999999998), (522.9999999999998, 320.9999999999999), (738.8285714285713, 366.99999999999983), (662.1142857142859, 383.0), (601.0000000000001, 383.0), (554.0, 383.0), (755.9999999999995, 398.9999999999997), (693.9999999999995, 429.99999999999994), (647.9999999999999, 445.91428571428554), (615.9999999999998, 460.9999999999998)]]
